Ranking system for Hyper Duplex Style Alloys

In [187]:
# Imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [188]:
# Mo Standard
Cr_values  = [27.0, 27.5, 28.0, 28.5, 29.0]
N_values   = [0.00, 0.05, 0.10, 0.15]
C_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.30, 0.50, 0.70, 0.80, 0.90, 1.00]
Mo_values = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

X_values   = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

In [189]:
# Input data 
X_elements = ["Ni", "Mn", "Si"]

service_temperature_C = 20

""" https://en.wikipedia.org/wiki/Prices_of_chemical_elements """
cost_usd_per_kg = {
    "Fe": 0.42,
    "C":  0.13,
    "Cr": 9.4,
    "Mo": 40.1,
    "Ni": 13.9,
    "Mn": 1.8,
    "Si": 1.7,
    "N":  0.15,
}

density_proxy = 7.8

In [190]:
# Ranking calculation proxies

def pren(Cr_wt, Mo_wt=0.0, N_wt=0.0, W_wt=0.0):
    """PREN ≈ %Cr + 3.3(%Mo + 0.5%W) + 16%N  (Si does not enter PREN)"""
    pren = Cr_wt + 3.3*(Mo_wt + 0.5*W_wt) + 16.0*N_wt
    return pren

# CHANGE FUNCTION
def strength_proxy(alloy):
    """Solid solution strengthening proxy (MPa)"""
    k = {"Cr": 35.0, "Mo": 120.0, "Ni": 20.0, "Mn": 25.0, "Si": 20.0, "C": 250.0, "N": 180.0}
    s = 0.0
    for el, wt in alloy.items():
        if el in k and wt > 0:
            s += k[el] * math.sqrt(wt)
    return s

def cost_proxy(alloy):
    """Cost proxy ($/kg)"""
    cost = 0.0
    for el, wt in alloy.items():
        el_cost = cost_usd_per_kg.get(el, 0.0)
        cost += (wt / 100.0) * el_cost
    return cost

In [191]:
# Generate data
rows = []
for X in X_elements:
    for C in C_values:
        for Cr in Cr_values:
            for N in N_values:
                for Mo in Mo_values:
                    for x_wt in X_values:
                        comp = {"C": C, "Cr": Cr, "N": N, "Mo": Mo, X: x_wt}
                        PREN = pren(Cr_wt=Cr, Mo_wt=comp.get("Mo", 0.0), N_wt=N)
                        dss  = strength_proxy(comp)

                        Fe_balance = 100.0 - (C + Cr + N + Mo + x_wt)

                        rows.append({
                            "X_element": X,
                            "C_wt%": C,
                            "Cr_wt%": Cr,
                            "N_wt%": N,
                            "Mo_wt%": Mo,
                            f"{X}_wt%": x_wt,
                            "Fe_wt% (balance)": Fe_balance,
                            "PREN": PREN,
                            "Strength_proxy_MPa": dss,
                            "Cost_proxy_$_per_kg": cost_proxy(comp),
                            "Service_T_C (info)": service_temperature_C,
                        })

df = pd.DataFrame(rows)

csv_path = "HD_Mo_Screening.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved: {csv_path}")
df.head(-10)

CSV saved: HD_Mo_Screening.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%
0,Ni,0.01,27.0,0.00,0.0,0.0,72.99,27.0,206.865335,2.538013,20,NaN,NaN
1,Ni,0.01,27.0,0.00,0.0,0.5,72.49,27.0,221.007470,2.607513,20,NaN,NaN
2,Ni,0.01,27.0,0.00,0.0,1.0,71.99,27.0,226.865335,2.677013,20,NaN,NaN
3,Ni,0.01,27.0,0.00,0.0,1.5,71.49,27.0,231.360232,2.746513,20,NaN,NaN
4,Ni,0.01,27.0,0.00,0.0,2.0,70.99,27.0,235.149606,2.816013,20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423345,Si,1.00,29.0,0.15,10.0,NaN,56.85,64.4,922.308804,6.788525,20,NaN,3.0
423346,Si,1.00,29.0,0.15,10.0,NaN,56.35,64.4,925.084362,6.797025,20,NaN,3.5
423347,Si,1.00,29.0,0.15,10.0,NaN,55.85,64.4,927.667788,6.805525,20,NaN,4.0
423348,Si,1.00,29.0,0.15,10.0,NaN,55.35,64.4,930.094195,6.814025,20,NaN,4.5


In [192]:
df["Score"] = df["PREN"] + df["Strength_proxy_MPa"] / 100.0
df_sorted = df.sort_values(by="Score", ascending=False)

csv_path2 = "HD_Mo_Ranking.csv"
df_sorted.to_csv(csv_path2, index=False)
print(f"CSV saved: {csv_path2}")
df_sorted.head(21)

CSV saved: HD_Mo_Ranking.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
282239,Mn,1.0,29.0,0.15,10.0,NaN,49.85,64.4,966.724729,6.917525,20,10.0,NaN,74.067247
282238,Mn,1.0,29.0,0.15,10.0,NaN,50.35,64.4,964.722963,6.908525,20,9.5,NaN,74.047230
282237,Mn,1.0,29.0,0.15,10.0,NaN,50.85,64.4,962.667788,6.899525,20,9.0,NaN,74.026678
282236,Mn,1.0,29.0,0.15,10.0,NaN,51.35,64.4,960.554686,6.890525,20,8.5,NaN,74.005547
282235,Mn,1.0,29.0,0.15,10.0,NaN,51.85,64.4,958.378466,6.881525,20,8.0,NaN,73.983785
282234,Mn,1.0,29.0,0.15,10.0,NaN,52.35,64.4,956.133107,6.872525,20,7.5,NaN,73.961331
273419,Mn,0.9,29.0,0.15,10.0,NaN,49.95,64.4,953.895554,6.917395,20,10.0,NaN,73.938956
282233,Mn,1.0,29.0,0.15,10.0,NaN,52.85,64.4,953.811570,6.863525,20,7.0,NaN,73.938116
273418,Mn,0.9,29.0,0.15,10.0,NaN,50.45,64.4,951.893787,6.908395,20,9.5,NaN,73.918938
282232,Mn,1.0,29.0,0.15,10.0,NaN,53.35,64.4,951.405532,6.854525,20,6.5,NaN,73.914055


Testing fixing certain compositions with singular variables

Fe 29Cr 10Mo 10Mn 0.7C varN

In [193]:
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.90) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 10) &
    np.isclose(df["Mn_wt%"], 10)
)

df_fixed = df.loc[mask].copy()
df_fixed["Score"] = df_fixed["PREN"] + df_fixed["Strength_proxy_MPa"] / 100.0
df_ranked = df_fixed.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path3 = "HD_Mo+Mn_Var_N.csv"
df_ranked.to_csv(csv_path3, index=False)
print(f"CSV saved: {csv_path3}")
df_ranked.head(21)

CSV saved: HD_Mo+Mn_Var_N.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.9,29.0,0.15,10.0,NaN,49.95,64.4,953.895554,6.917395,20,10.0,NaN,73.938956
1,Mn,0.9,29.0,0.10,10.0,NaN,50.00,63.6,941.102851,6.917320,20,10.0,NaN,73.011029
2,Mn,0.9,29.0,0.05,10.0,NaN,50.05,62.8,924.431077,6.917245,20,10.0,NaN,72.044311
3,Mn,0.9,29.0,0.00,10.0,NaN,50.10,62.0,884.181853,6.917170,20,10.0,NaN,70.841819


Fe 29Cr 10Mn 0.7C varMo
$\newline$
No N

In [194]:
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.90) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mn_wt%"], 10) &
    np.isclose(df["N_wt%"], 0.0) &
    df["Mo_wt%"].between(0.0, 10.0)
)

df_mo_sweep2 = df.loc[mask].copy()
df_mo_sweep2["Score"] = df_mo_sweep2["PREN"] + df_mo_sweep2["Strength_proxy_MPa"] / 100.0
df_mo_sweep2 = df_mo_sweep2.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path4 = "HD_Mo_0N_Var_Mo.csv"
df_mo_sweep2.to_csv(csv_path4, index=False)
print(f"CSV saved: {csv_path4}")
df_mo_sweep2.head(21)


CSV saved: HD_Mo_0N_Var_Mo.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.9,29.0,0.0,10.0,NaN,50.1,62.00,884.181853,6.91717,20,10.0,NaN,70.841819
1,Mn,0.9,29.0,0.0,9.5,NaN,50.6,60.35,874.573374,6.71667,20,10.0,NaN,69.095734
2,Mn,0.9,29.0,0.0,9.0,NaN,51.1,58.70,864.708534,6.51617,20,10.0,NaN,67.347085
3,Mn,0.9,29.0,0.0,8.5,NaN,51.6,57.05,854.565648,6.31567,20,10.0,NaN,65.595656
4,Mn,0.9,29.0,0.0,8.0,NaN,52.1,55.40,844.119789,6.11517,20,10.0,NaN,63.841198
5,Mn,0.9,29.0,0.0,7.5,NaN,52.6,53.75,833.342069,5.91467,20,10.0,NaN,62.083421
6,Mn,0.9,29.0,0.0,7.0,NaN,53.1,52.10,822.198692,5.71417,20,10.0,NaN,60.321987
7,Mn,0.9,29.0,0.0,6.5,NaN,53.6,50.45,810.649705,5.51367,20,10.0,NaN,58.556497
8,Mn,0.9,29.0,0.0,6.0,NaN,54.1,48.80,798.647303,5.31317,20,10.0,NaN,56.786473
9,Mn,0.9,29.0,0.0,5.5,NaN,54.6,47.15,786.133480,5.11267,20,10.0,NaN,55.011335


Fe 29Cr 10Mo 0.7C varMn
$\newline$
No N

In [195]:
import numpy as np

# --- filter: fixed C, Cr, Mn; N = 0; Mo between 5.0 and 10.5 ---
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.90) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 10) &
    np.isclose(df["N_wt%"], 0.0) &
    df["Mn_wt%"].between(0.0, 10.0)
)

df_mn_sweep = df.loc[mask].copy()
df_mn_sweep["Score"] = df_mn_sweep["PREN"] + df_mn_sweep["Strength_proxy_MPa"] / 100.0
df_mn_sweep = df_mn_sweep.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path4 = "HD_Mo_0N_Var_Mn.csv"
df_mn_sweep.to_csv(csv_path4, index=False)
print(f"CSV saved: {csv_path4}")
df_mn_sweep.head(21)

CSV saved: HD_Mo_0N_Var_Mn.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.9,29.0,0.0,10.0,NaN,50.1,62.0,884.181853,6.91717,20,10.0,NaN,70.841819
1,Mn,0.9,29.0,0.0,10.0,NaN,50.6,62.0,882.180087,6.90817,20,9.5,NaN,70.821801
2,Mn,0.9,29.0,0.0,10.0,NaN,51.1,62.0,880.124912,6.89917,20,9.0,NaN,70.801249
3,Mn,0.9,29.0,0.0,10.0,NaN,51.6,62.0,878.011811,6.89017,20,8.5,NaN,70.780118
4,Mn,0.9,29.0,0.0,10.0,NaN,52.1,62.0,875.835590,6.88117,20,8.0,NaN,70.758356
5,Mn,0.9,29.0,0.0,10.0,NaN,52.6,62.0,873.590232,6.87217,20,7.5,NaN,70.735902
6,Mn,0.9,29.0,0.0,10.0,NaN,53.1,62.0,871.268695,6.86317,20,7.0,NaN,70.712687
7,Mn,0.9,29.0,0.0,10.0,NaN,53.6,62.0,868.862656,6.85417,20,6.5,NaN,70.688627
8,Mn,0.9,29.0,0.0,10.0,NaN,54.1,62.0,866.362156,6.84517,20,6.0,NaN,70.663622
9,Mn,0.9,29.0,0.0,10.0,NaN,54.6,62.0,863.755109,6.83617,20,5.5,NaN,70.637551


Fe 29Cr 10Mo 10Mn varC
$\newline$
No N

In [196]:
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["Mn_wt%"], 9.5) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 9.5) &
    np.isclose(df["N_wt%"], 0.0)
)

df_C_sweep = df.loc[mask].copy()
df_C_sweep["Score"] = df_C_sweep["PREN"] + df_C_sweep["Strength_proxy_MPa"] / 100.0
df_C_sweep = df_C_sweep.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path4 = "HD_Mo_0N_Var_C.csv"
df_C_sweep.to_csv(csv_path4, index=False)
print(f"CSV saved: {csv_path4}")
df_C_sweep.head(21)

CSV saved: HD_Mo_0N_Var_C.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,1.00,29.0,0.0,9.5,NaN,51.00,60.35,885.400783,6.707800,20,9.5,NaN,69.204008
1,Mn,0.90,29.0,0.0,9.5,NaN,51.10,60.35,872.571608,6.707670,20,9.5,NaN,69.075716
2,Mn,0.80,29.0,0.0,9.5,NaN,51.20,60.35,859.007581,6.707540,20,9.5,NaN,68.940076
3,Mn,0.70,29.0,0.0,9.5,NaN,51.30,60.35,844.565790,6.707410,20,9.5,NaN,68.795658
4,Mn,0.50,29.0,0.0,9.5,NaN,51.50,60.35,812.177479,6.707150,20,9.5,NaN,68.471775
5,Mn,0.30,29.0,0.0,9.5,NaN,51.70,60.35,772.331423,6.706890,20,9.5,NaN,68.073314
6,Mn,0.10,29.0,0.0,9.5,NaN,51.90,60.35,714.457725,6.706630,20,9.5,NaN,67.494577
7,Mn,0.09,29.0,0.0,9.5,NaN,51.91,60.35,710.400783,6.706617,20,9.5,NaN,67.454008
8,Mn,0.08,29.0,0.0,9.5,NaN,51.92,60.35,706.111462,6.706604,20,9.5,NaN,67.411115
9,Mn,0.07,29.0,0.0,9.5,NaN,51.93,60.35,701.544566,6.706591,20,9.5,NaN,67.365446


In [197]:
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 8.0) &
    np.isclose(df["Mn_wt%"], 0.0)
)

df_final_sweep = df.loc[mask].copy()
df_final_sweep["Score"] = df_final_sweep["PREN"] + df_final_sweep["Strength_proxy_MPa"] / 100.0
df_final_sweep = df_final_sweep.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path4 = "HD_8Mo_Var_N.csv"
df_final_sweep.to_csv(csv_path4, index=False)
print(f"CSV saved: {csv_path4}")
df_final_sweep.head(60)

CSV saved: HD_8Mo_Var_N.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,1.00,29.0,0.15,8.0,NaN,61.85,57.8,847.605723,5.935525,20,0.0,NaN,66.276057
1,Mn,0.90,29.0,0.15,8.0,NaN,61.95,57.8,834.776548,5.935395,20,0.0,NaN,66.147765
2,Mn,0.80,29.0,0.15,8.0,NaN,62.05,57.8,821.212521,5.935265,20,0.0,NaN,66.012125
3,Mn,0.70,29.0,0.15,8.0,NaN,62.15,57.8,806.770730,5.935135,20,0.0,NaN,65.867707
4,Mn,0.50,29.0,0.15,8.0,NaN,62.35,57.8,774.382419,5.934875,20,0.0,NaN,65.543824
5,Mn,1.00,29.0,0.10,8.0,NaN,61.90,57.0,834.813021,5.935450,20,0.0,NaN,65.348130
6,Mn,0.90,29.0,0.10,8.0,NaN,62.00,57.0,821.983846,5.935320,20,0.0,NaN,65.219838
7,Mn,0.30,29.0,0.15,8.0,NaN,62.55,57.8,734.536363,5.934615,20,0.0,NaN,65.145364
8,Mn,0.80,29.0,0.10,8.0,NaN,62.10,57.0,808.419819,5.935190,20,0.0,NaN,65.084198
9,Mn,0.70,29.0,0.10,8.0,NaN,62.20,57.0,793.978028,5.935060,20,0.0,NaN,64.939780


Fe 29Cr 6.5Mo 9.5Mn 0.7C varN

In [198]:
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.70) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 6.5) &
    np.isclose(df["Mn_wt%"], 9.5)
)

# --- apply filter, (re)compute score, sort, show ---
df_fixed = df.loc[mask].copy()
df_fixed["Score"] = df_fixed["PREN"] + df_fixed["Strength_proxy_MPa"] / 100.0
df_ranked = df_fixed.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path5 = "HD_Mo_6.5Mo_Var_N.csv"
df_ranked.to_csv(csv_path5, index=False)
print(f"CSV saved: {csv_path5}")
df_ranked.head(21)

CSV saved: HD_Mo_6.5Mo_Var_N.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.7,29.0,0.15,6.5,NaN,54.15,52.85,850.355821,5.504635,20,9.5,NaN,61.353558
1,Mn,0.7,29.0,0.10,6.5,NaN,54.20,52.05,837.563119,5.504560,20,9.5,NaN,60.425631
2,Mn,0.7,29.0,0.05,6.5,NaN,54.25,51.25,820.891344,5.504485,20,9.5,NaN,59.458913
3,Mn,0.7,29.0,0.00,6.5,NaN,54.30,50.45,780.642121,5.504410,20,9.5,NaN,58.256421


In [199]:
# CONSIDER CPT (Critical Pitting Temperature)
# CPT = PREN x 2.5 -60 (can be -10/20 more)